In [1]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots, CSV, Serialization
using JuMP, Ipopt
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [6]:
deltas = deserialize("deltas.jls");
df = deserialize("df.jls");
dfj = deserialize("dfj.jls");

In [13]:
# Data
# linear characteristics (staffing variables):
X1 = hcat(df.x1, df.x2); 
# non-linear characteristics:
D = hcat(df.d, df.d2); 
X2 = D
Z = X1 # TODO: verify this

T = df.t; #tract IDs
J = df.j; #facility IDs
M = df.M; #tract populations
q = dfj.q; #one quantity per facility


# RC draws
nI = 1000; #number of draws
K = 2; #number of non-linear characteristics 
ζ = rand(Normal(0,1), K, nI);


In [11]:
# # disutility of distance
# β = [-0.5, -1];

once you find the correct mean utilities for a resonable distnace parameter, use that as the starting point for the fixed point the next time

In [21]:
Φ1 = Z' * Z
initial_b_guess = [-1., -2.]

b_init = optimize(β -> m.gmm(β, q, X1, D, X1, M, ζ, J, T, Φ1), initial_b_guess, LBFGS()).minimizer 
# b_init := params_2 = res_init_wt.x

iterations: 644
dist: 9.938716516444401e-13


iterations: 644
dist: 9.938716516444401e-13


iterations: 644
dist: 9.920952948050399e-13


iterations: 644
dist: 9.938716516444401e-13


In [ ]:
b_init

In [ ]:

# d_init = m.compute_deltas(q, D, M, ζ, b_init, J, T)

In [ ]:
b_ = optimize(β -> m.gmm(β, q, X, Z, ζ, Φ1), initial_b_guess, LBFGS()).minimizer
d_ = m.compute_deltas(q, D, M, ζ, b_, J, T)

In [ ]:
b_forΦ = m.grad(X, Z, Φ1, d_)

In [ ]:
ξ_ = d_ - X * b_forΦ;
Φ = Z' * ξ_ * ξ_' * Z

In [ ]:
b = optimize(x -> m.gmm(x, s, X, Z, ζ, Φ), initial_b_guess, LBFGS()).minimizer

In [ ]:
δ = m.loop(s, X, ζ, b, verbose = true);

In [ ]:
β_blp = m.grad(X, Z, Φ, δ)

In [ ]:
# m.gmm(β, dfj.s, X, Z, ζ)

# (X' * X )*( X'  * deltas)
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# @NLobjective(model, Min, m.gmm(β, dfj.s, X, Z, ζ))

# this is messed up
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# register(model, :g, 1, m.gmm, m.grad, autodiff = true)
# @NLobjective(model, Min, (s, X, Z, ζ))
# optimize!(model)
# solution_summary(model; verbose = true)

In [ ]:
K = 2
nI = 1000
J = nrow(dfj)
X = hcat(dfj.auxd, dfj.auxd2);
println(size(X))
ζ = rand(Normal(0,1), K, nI);
println(size(ζ))
s = dfj.s; #using shares or quantities

In [ ]:
δ0 = zeros(1, size(ζ, 2))
δ_ = ones(size(s))
β = [-0.5, -1.];


In [ ]:
println(size(ζ .* β))

In [ ]:
J

In [ ]:
println("Dims of ζ: ", size(ζ))
println("Dims of ζ .* β: ", size(ζ .* β))
println("Dims of X * (ζ .* β): ", size(X * (ζ .* β)))
println("Dims of δ_ .+ (X * (ζ .* β)): ", size(δ_ .+ (X * (ζ .* β))))
println("Dims of u: ", size(u))
println("Dims of s: ", size(s))

In [ ]:
ζ

In [ ]:
β

In [ ]:
(ζ .* β)

In [ ]:
X

In [ ]:
(X * (ζ .* β))

In [ ]:
u = [δ_ .+ (X * (ζ .* β)); δ0]
# println(mean(u))
e = exp.(u)
s = mean(e, dims=2)